In [1]:
import pickle
import re

In [2]:
import pandas as pd

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
with open('category.pickle', 'rb') as fr:
    category = pickle.load(fr)

In [5]:
len(category)

470

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'}
basic_url = "https://www.yes24.com"

In [7]:
def get_basic_info(page_url):
    print(f'Get basic information in {page_url}')
    res = requests.get(page_url, headers= headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    boxes = soup.select('div.cCont_goodsSet')

    basic_info_list = []
    for i, box in enumerate(boxes):
        # title box
        name_box = box.select_one('div.goods_name')

        # title, url
        title_box = name_box.select_one('a')
        title_text = title_box.text.strip()
        book_url = basic_url + title_box['href']

        # full_name
        name_front = name_box.select_one('span.gd_nameF').text
        name_end = name_box.select_one('span.gd_nameE').text
        name_feature = name_box.select_one('span.gd_feature').text
        full_name = ' '.join([name_front, title_text, name_end, name_feature])

        # pub_box
        pub_box = box.select_one('div.goods_pubGrp')
        authors = pub_box.select_one('span.goods_auth').text.strip()
        # $$$ 저자, 역자 나누기
        # $$$ 정보 더 보기/감추기 <- 클렌징
        publisher = pub_box.select_one('span.goods_pub').text

        # print(f"title: [{title_text}]")
        # print("full name:", full_name)
        # print('url:', book_url)
        # print('authors:', authors)
        # print('publisher:', publisher)
        # print('*'*10+'\n')

        basic_info_list.append([title_text, full_name, book_url, authors, publisher])
    return basic_info_list

In [ ]:
quantity = []

for cate_name, cate_url in category.items():
    # 총 페이지 수 구하기
    res = requests.get(cate_url, headers= headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    max_page_num = int(re.search(r'\d+$', soup.select_one('div.yesUI_pagenS>a.end')['href']).group(0))

    quantity.append([cate_name, max_page_num, cate_url])
    # print(basic_infos)

    print(cate_name, max_page_num, cate_url)

# 저장
df = pd.DataFrame(quantity, columns=['cate_name', 'max_page_num', 'cate_url'])
df.to_csv('quantity.csv')

In [9]:
for cate_name, cate_url in category.items():
    print(cate_url)

    # 총 페이지 수 구하기
    res = requests.get(cate_url, headers= headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    max_page_num = int(re.search(r'\d+$', soup.select_one('div.yesUI_pagenS>a.end')['href']).group(0))
    print(max_page_num)

    basic_infos = []
    # 각 페이지에 대하여 기본 정보 스크래핑
    page_url = cate_url+"?PageNumber={}"
    for i in range(1, max_page_num+1):
        basic_infos.extend(get_basic_info(page_url.format(i)))
        break
    # print(basic_infos)

    # 저장
    
    break

https://www.yes24.com/24/Category/Display/001001025007004004
199
Get basic information in https://www.yes24.com/24/Category/Display/001001025007004004?PageNumber=1


In [10]:
df = pd.DataFrame(basic_infos, columns=['title_text', 'full_name', 'book_url', 'authors', 'publisher'])
df

,title_text,full_name,book_url,authors,publisher
0,트렌드 코리아 2025,트렌드 코리아 2025 2025 대한민국 소비트렌드 전망,https://www.yes24.com//Product/Goods/133213071,"김난도,전미영,최지혜,권정윤,한다혜,이혜원,이준영,이향은,추예린,전다현 저",미래의창
1,시대예보 : 호명사회,시대예보 : 호명사회,https://www.yes24.com//Product/Goods/133236068,송길영 저,교보문고
2,피벗의 시대 2025년 경제전망,피벗의 시대 2025년 경제전망 2025년을 결정지을 18대 경제트렌드,https://www.yes24.com//Product/Goods/133233432,김광석 저,이든하우스
3,머니 트렌드 2025,머니 트렌드 2025 새로운 부의 기회를 선점할 55가지 성공 시나리오 [ 예...,https://www.yes24.com//Product/Goods/132970116,"정태익,김도윤,김광석,김용섭,김현준,정수종,채상욱,최재붕 저",북모먼트
4,시대예보: 핵개인의 시대,시대예보: 핵개인의 시대 [ 양장 ],https://www.yes24.com//Product/Goods/122457906,송길영 저,교보문고
5,트렌드 코리아 2024,트렌드 코리아 2024 청룡을 타고 비상하는 2024를 기원하며!,https://www.yes24.com//Product/Goods/122426425,"김난도,전미영,최지혜,이수진,권정윤,한다혜,이준영,이향은,이혜원,추예린 저 외 1명...",미래의창
6,AI 반도체 혁명,"AI 반도체 혁명 3차 반도체 전쟁, 최후의 승자는 누가 될 것인가",https://www.yes24.com//Product/Goods/132616188,"권순우,이동수,권세중,유지원 저",페이지2북스
7,박태웅의 AI 강의 2025,박태웅의 AI 강의 2025 인공지능의 출현부터 일상으로의 침투까지 우리와 미래를...,https://www.yes24.com//Product/Goods/133308510,박태웅 저,한빛비즈
8,챗봇 2025 트렌드&활용백과,챗봇 2025 트렌드&활용백과,https://www.yes24.com//Product/Goods/126870580,"김덕진,서승완 저",스마트북스
9,넛지: 파이널 에디션,"넛지: 파이널 에디션 복잡한 세상에서 똑똑한 선택을 이끄는 힘 [ 양장, 개정판 ]",https://www.yes24.com//Product/Goods/110157236,"리처드 탈러,캐스 선스타인 저/이경식 역/최정규 감수",리더스북


In [11]:
df.to_csv('basic_sample.csv')